<a href="https://colab.research.google.com/github/AlexArutiunian/freelance_practice/blob/main/build_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Суть задачи: собрать с сайта нормативно правовых актов https://docs.cntd.ru

1) Названия НПА

2) Ссылки на их содержимое

3) Описание объектов учёта (выделить в содержимом)

После этого проанализировать собранные данные на предмет схожих и отличительных особенностей

Создание списка регионов для формирования запросов на сайте в виде

f"https://docs.cntd.ru/search?q=Строительный%20Контроль%20{region}"

где region - это элемент списка regions, предоставленного в качестве входных данных.

In [ ]:
import pandas as pd

excel_file = 'regions.xlsx'
column_name = 'Регионы'

df = pd.read_excel(excel_file)

regions = df[column_name].tolist()
print(regions)

['Москва', 'Санкт-Петербург', 'Московская область', 'Пермский край', 'Самарская область', 'Нижегородская область', 'Новосибирская область', 'Краснодарский край', 'Ямало-Ненецкий АО', 'Приморский край', 'Волгоградская область', 'Республика Саха (Якутия)', 'Республика Татарстан', 'Свердловская область', 'Кемеровская область', 'Тюменская область', 'Ленинградская область', 'Мурманская область', 'Красноярский край', 'ХМАО-Югра', 'Республика Коми', 'Сахалинская область', 'Иркутская область', 'Воронежская область', 'Тульская область', 'Челябинская область', 'Республика Башкортостан', 'Белгородская область', 'Вологодская область', 'Архангельская область', 'Калужская область', 'Калининградская область', 'Оренбургская область', 'Ростовская область', 'Хабаровский край', 'Смоленская область', 'Ярославская область', 'Курская область', 'Республика Удмуртия', 'Липецкая область', 'Республика Крым', 'Кировская область', 'Республика Бурятия', 'Омская область', 'Саратовская область', 'Республика Калмыкия

Формирование запросов и сбор ссылок, названий строительных НПА

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import HTML

def parc(url):
  response = requests.get(url)
  page_content = response.text

  # Создать объект BeautifulSoup для парсинга страницы
  soup = BeautifulSoup(page_content, 'html.parser')

  # Какие именно теги и классы нужно находить - узнаём,
  # анализируя код страницы через инспектор кода

  # Найти все элементы li с классом document-list_i
  elements = soup.find_all('li', class_='document-list_i')
  # списки для хранения данных
  links = []
  text_data = []

  # Проход по каждому найденному элементу
  # ИЗМЕНИТЬ В СЛУЧАЕ ПАРСИНГА ПО ДРУГОМУ САЙТУ ДОКУМЕНТОВ
  for element in elements:
      # Найти тег svg
      svg = element.find('svg')

      # Проверить условие title = "Действующий"
      if svg and svg.get('title') == 'Действующий':
          # Найти ссылку в теге a
          link = element.find('a')['href']
          links.append("https://docs.cntd.ru" + link)

          # Найти текст в теге li
          text = element.text.strip()
          text_data.append(text)

      if(len(links) == 6):
        break
  return links, text_data


dfs = []  # Список для хранения DataFrame

# regions - использовать либо кастомный,
# либо тот, что был сформирован в первой ячейке
regions = ["Общие требования по России"]
for region in regions:
    url = f"https://docs.cntd.ru/search?q=Строительный%20Контроль%20{region}"
    url2 = f"https://docs.cntd.ru/search?q=Строительный%20Надзор%20{region}"

    parc_data_1 = parc(url)
    parc_data_2 = parc(url2)

    max_length = max(len(parc_data_1[1]), len(parc_data_2[1]))

    # Добавляем пустые строки, чтобы сделать списки одинаковой длины
    # И не получать исключение, которое отлавливается ниже
    while max_length - len(parc_data_1[1]):
       parc_data_1[1].append([''])
    while max_length - len(parc_data_1[0]):
       parc_data_1[0].append([''])

    while max_length - len(parc_data_2[0]):
       parc_data_2[0].append([''])
    while max_length - len(parc_data_2[1]):
       parc_data_2[1].append([''])


    links1 = []
    for elem in parc_data_1[0]:
      elem = f"<a href='{elem}'>{elem}</a>"
      links1.append(elem)

    links2 = []
    for elem in parc_data_2[0]:
      elem = f"<a href='{elem}'>{elem}</a>"
      links2.append(elem)

    data = {'Регион': region,
            'Название НПА по стройконтролю': parc_data_1[1],
            'Ссылка на НПА по стройконтролю': links1,
            'Название НПА по стройнадзору': parc_data_2[1],
            'Ссылка на НПА по стройнадзору': links2}
    print(data)
    try:
        df = pd.DataFrame(data)
        dfs.append(df)
    except Exception as e:
        print(e)
        print(region)


c_df = pd.concat(dfs, ignore_index=True)
# Разделить массивы на отдельные строки
# Создать HTML-код таблицы с кликабельными ссылками
html_table = c_df.to_html(escape=False)


# Отображение HTML-кода таблицы с кликабельными ссылками
display(HTML(html_table))

excel_file = 'general_.xlsx'
c_df.to_excel(excel_file)
print('Таблица успешно сохранена в Excel!')
excel_file = 'general_.xlsx'

{'Регион': 'Общие требования по России', 'Название НПА по стройконтролю': ['Об утверждении общих требований к организации и осуществлению регионального государственного строительного надзора, внесении изменений в постановление Правительства Российской Федерации от 30 июня 2021 г. N 1087 и признании утратившими силу некоторых актов Правительства Российской Федерации (с изменениями на 30 марта 2023 года) \n        Постановление Правительства РФ от 01 декабря 2021  г. № 2161', [''], [''], [''], [''], ['']], 'Ссылка на НПА по стройконтролю': ["<a href='https://docs.cntd.ru/document/727294967'>https://docs.cntd.ru/document/727294967</a>", "<a href='['']'>['']</a>", "<a href='['']'>['']</a>", "<a href='['']'>['']</a>", "<a href='['']'>['']</a>", "<a href='['']'>['']</a>"], 'Название НПА по стройнадзору': ['Об утверждении общих требований к организации и осуществлению регионального государственного строительного надзора, внесении изменений в постановление Правительства Российской Федерации от

,Регион,Название НПА по стройконтролю,Ссылка на НПА по стройконтролю,Название НПА по стройнадзору,Ссылка на НПА по стройнадзору
0,Общие требования по России,"Об утверждении общих требований к организации и осуществлению регионального государственного строительного надзора, внесении изменений в постановление Правительства Российской Федерации от 30 июня 2021 г. N 1087 и признании утратившими силу некоторых актов Правительства Российской Федерации (с изменениями на 30 марта 2023 года) \n Постановление Правительства РФ от 01 декабря 2021 г. № 2161",https://docs.cntd.ru/document/727294967,"Об утверждении общих требований к организации и осуществлению регионального государственного строительного надзора, внесении изменений в постановление Правительства Российской Федерации от 30 июня 2021 г. N 1087 и признании утратившими силу некоторых актов Правительства Российской Федерации (с изменениями на 30 марта 2023 года) \n Постановление Правительства РФ от 01 декабря 2021 г. № 2161",https://docs.cntd.ru/document/727294967
1,Общие требования по России,[],[''],"Об утверждении Требований к оформлению протоколов общих собраний собственников помещений в многоквартирных домах и Порядка направления подлинников решений и протоколов общих собраний собственников помещений в многоквартирных домах в уполномоченные органы исполнительной власти субъектов Российской Федерации, осуществляющие государственный жилищный надзор (с изменениями на 16 сентября 2022 года) \n Приказ Министерства строительства и жилищно-коммунального хозяйства Российской Федерации от 28 января 2019 г. № 44/пр",https://docs.cntd.ru/document/552449986
2,Общие требования по России,[],[''],"Об определении размера активов кредитного потребительского кооператива, общее число членов которого не превышает три тысячи физических и (или) юридических лиц, при превышении которого Банк России осуществляет надзор за соблюдением требований законодательства Российской Федерации о кредитной кооперации \n Указание Банка России от 09 декабря 2020 г. № 5654-У",https://docs.cntd.ru/document/573481367
3,Общие требования по России,[],[''],"Об определении размера активов сельскохозяйственного кредитного потребительского кооператива, общее число членов и ассоциированных членов которого не превышает три тысячи физических лиц и (или) юридических лиц, при превышении которого Банк России осуществляет надзор за соблюдением требований законодательства Российской Федерации о сельскохозяйственной кооперации \n Указание Банка России от 09 декабря 2020 г. № 5655-У",https://docs.cntd.ru/document/573481366
4,Общие требования по России,[],[''],"Об утверждении Административного регламента по исполнению Министерством строительства и жилищно-коммунального хозяйства Российской Федерации государственной функции по осуществлению контроля за соблюдением органами исполнительной власти субъектов Российской Федерации, осуществляющими региональный государственный жилищный надзор, требований к лицензированию предпринимательской деятельности по управлению многоквартирными домами \n Приказ Министерства строительства и жилищно-коммунального хозяйства Российской Федерации от 30 августа 2017 г. № 1186/пр",https://docs.cntd.ru/document/542608369
5,Общие требования по России,[],[''],"Об установлении требований к формату предоставления заключения уполномоченного на осуществление государственного контроля (надзора) в области долевого строительства многоквартирных домов и (или) иных объектов недвижимости органа исполнительной власти субъекта Российской Федерации, на территории которого осуществляется строительство (создание) соответствующих многоквартирного дома и (или) иного объекта недвижимости, о соответствии застройщика и проектной декларации требованиям Федерального закона от 30.12.2004 N 214-ФЗ ""Об участии в долевом строительстве многоквартирных домов и иных объектов недвижимости и о внесении изменений в некоторые законодательные акты Российской Федерации"" \n Приказ Росреестра от 16 апреля 2019 г. № П/0149",https://docs.cntd.ru/doc

Таблица успешно сохранена в Excel!


In [ ]:
# Вывести таблицу в ячейке Google Colab
c_df

Поиск предложений с описанием объектов строительного надзора и учёта.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import HTML

# Считать таблицу из файла Excel
excel_file = "outtest.xlsx"
read_df = pd.read_excel(excel_file)
list_1 = read_df['Ссылка на НПА по стройконтролю']

def find_sentences_with_word(text, word):
    sentences = text.split('.')
    found_sentences = []

    for sentence in sentences:
        words = sentence.strip().split()
        for i, w in enumerate(words):
            if word.lower() in w.lower():
                start_index = max(0, i - 10)
                end_index = min(i + 10, len(words))
                found_phrase = ' '.join(words[start_index:end_index])
                found_sentences.append(found_phrase.strip() + "\n")

    found_sentences = list(set(found_sentences))  # Remove duplicates

    return found_sentences

# Создать список для хранения предложений
found_sentences = []

for index, elem in enumerate(list_1):
    try:
      response = requests.get(elem)
      page_content = response.text


      soup = BeautifulSoup(page_content, 'html.parser')

      content = soup.find("div", class_="document-content").get_text()
      if(index == 0):
        with open("test.txt", "w") as f:
          f.writelines(content)
        print(content)
      sentences = find_sentences_with_word(content, 'объект')
      print(sentences)
      read_df.at[index, 'объекты учета (стройконтроль)'] = ';'.join(sentences)
    except Exception as e:
      print(f"{e}")

# Отобразить исходную таблицу с новым столбцом
display(read_df)

# Создать новый DataFrame для найденных предложений
sentences_df = pd.DataFrame({'Предложения со словом "объект"': found_sentences})

# Отобразить таблицу в формате pandas
html_table = sentences_df.to_html(escape=False)

# Отображение HTML-кода таблицы с кликабельными ссылками
display(HTML(html_table))

excel_file = 'outtest.xlsx'
read_df.to_excel(excel_file)
print('Таблица успешно сохранена в Excel!')
excel_file = 'output.xlsx'

        Правительство МосквыКОМИТЕТ ГОРОДА МОСКВЫ ПО ЦЕНОВОЙ ПОЛИТИКЕ В СТРОИТЕЛЬСТВЕ И ГОСУДАРСТВЕННОЙ ЭКСПЕРТИЗЕ ПРОЕКТОВПРИКАЗот 20 марта 2014 года N 29Об утверждении Порядка определения нормативов затрат на содержание службы заказчика и осуществление строительного контроля(с изменениями на 28 января 2022 года)Информация об изменяющих документах____________________________________________________________________ Документ с изменениями, внесенными: приказом Москомэкспертизы от 29 августа 2014 года N 80; приказом Москомэкспертизы от 15 декабря 2017 года N МКЭ-ОД/17-69; приказом Москомэкспертизы от 11 марта 2019 года N МКЭ-ОД/19-14; приказом Москомэкспертизы от 30 декабря 2019 года N МКЭ-ОД/19-95; приказом Москомэкспертизы от 4 февраля 2021 года N МКЭ-ОД/21-7; приказом Москомэкспертизы от 28 января 2022 года N МКЭ-ОД/22-11. ____________________________________________________________________На основании пунктов 4.2.6, 4.2.13 Положения о Комитете города Москвы по ценовой политике в стро

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Регион,Название НПА по стройконтролю,Ссылка на НПА по стройконтролю,Название НПА по стройнадзору,Ссылка на НПА по стройнадзору,объекты учета (стройконтроль),объекты учета (стройнадзор)
0,0,0,0,0,Москва,Об утверждении Порядка определения нормативов ...,https://docs.cntd.ru/document/537952547,Об утверждении Административного регламента пр...,https://docs.cntd.ru/document/456026260,и осуществление строительного контроля при стр...,планировки территории и проект межевания терри...
1,1,1,1,1,Москва,"Об автоматизированной информационной системе ""...",https://docs.cntd.ru/document/537965372,NaN,NaN,осуществляется с привлечением денежных средств...,планировки территории и проект межевания терри...
2,2,2,2,2,Москва,О создании Комитета города Москвы по обеспечен...,https://docs.cntd.ru/document/537904197,Об утверждении Положения о Комитете государств...,https://docs.cntd.ru/document/537905970,,строительства или садового дома на земельном у...
3,3,3,3,3,Москва,Об утверждении Положения о региональном госуда...,https://docs.cntd.ru/document/727795406,Об утверждении Положения о региональном госуда...,https://docs.cntd.ru/document/727654384,(надзоре) в области долевого строительства мно...,Объектами регионального государственного строи...
4,4,4,4,4,Москва,"О введении ""Системы регулирования, учета и кон...",https://docs.cntd.ru/document/8318960,О Комитете государственного строительного надз...,https://docs.cntd.ru/document/3663509,"и ведомственной принадлежности, предусмотренны...",на осуществление государственного строительног...
...,...,...,...,...,...,...,...,...,...,...,...
491,491,491,491,491,Еврейская АО,О признании утратившими силу некоторых приказо...,https://docs.cntd.ru/document/578111001,Об утверждении Положения об инспекции государс...,https://docs.cntd.ru/document/460153345,в области долевого строительства многоквартирн...,(надзора) в области долевого строительства мно...
492,492,492,492,492,Еврейская АО,Об утверждении перечня показателей результатив...,https://docs.cntd.ru/document/574735072,Об общественном совете при инспекции государст...,https://docs.cntd.ru/document/570715382,выполнения работ и применяемых строительных ма...,NaN
493,493,493,493,493,Еврейская АО,О внесении изменений и дополнений в приказ упр...,https://docs.cntd.ru/document/444924942,Об утверждении перечня показателей результатив...,https://docs.cntd.ru/document/574735072,(надзора) в области долевого строительства мно...,выполнения работ и применяемых строительных ма...
494,494,494,494,494,Еврейская АО,О ВНЕСЕНИИ ИЗМЕНЕНИЙ В ПОСТАНОВЛЕНИЕ ПРАВИТЕЛЬ...,https://docs.cntd.ru/document/412302781,О внесении изменений в Перечень показателей ре...,https://docs.cntd.ru/document/561614079,и надзора в области долевого строительства мно...,выполнения работ и применяемых строительных ма...


,"Предложения со словом ""объект"""


Таблица успешно сохранена в Excel!


In [ ]:
def find_sentences_with_word(text, word):
    sentences = text.split('.')
    found_sentences = []

    for sentence in sentences:
        words = sentence.strip().split()
        for i, w in enumerate(words):
            if word.lower() in w.lower():
                start_index = max(0, i - 12)
                end_index = min(i + 12, len(words))
                found_phrase = ' '.join(words[start_index:end_index])
                found_sentences.append(found_phrase.strip() + "\n")

    found_sentences = list(set(found_sentences))  # Remove duplicates

    return found_sentences

text = '         Правительство МосквыКОМИТЕТ ГОРОДА МОСКВЫ ПО ЦЕНОВОЙ ПОЛИТИКЕ В СТРОИТЕЛЬСТВЕ И ГОСУДАРСТВЕННОЙ ЭКСПЕРТИЗЕ ПРОЕКТОВПРИКАЗот 20 марта 2014 года N 29Об утверждении Порядка определения нормативов затрат на содержание службы заказчика и осуществление строительного контроля(с изменениями на 28 января 2022 года)Информация об изменяющих документах____________________________________________________________________ Документ с изменениями, внесенными: приказом Москомэкспертизы от 29 августа 2014 года N 80; приказом Москомэкспертизы от 15 декабря 2017 года N МКЭ-ОД/17-69; приказом Москомэкспертизы от 11 марта 2019 года N МКЭ-ОД/19-14; приказом Москомэкспертизы от 30 декабря 2019 года N МКЭ-ОД/19-95; приказом Москомэкспертизы от 4 февраля 2021 года N МКЭ-ОД/21-7; приказом Москомэкспертизы от 28 января 2022 года N МКЭ-ОД/22-11. ____________________________________________________________________На основании пунктов 4.2.6, 4.2.13 Положения о Комитете города Москвы по ценовой политике в строительстве и государственной экспертизе проектов, утвержденного постановлением Правительства Москвы от 24 февраля 2011 года N 48-ПП,приказываю:1. Утвердить Порядок определения нормативов затрат на содержание службы заказчика и осуществление строительного контроля при строительстве, реконструкции или капитальном ремонте объектов капитального строительства, финансирование которых осуществляется с привлечением средств бюджета города Москвы (приложение).2. Настоящий приказ вступает в силу с 1 апреля 2014 года.3. Установить, что утвержденная до 1 апреля 2014 года в установленном порядке проектная документация пересмотру и переутверждению в связи с выходом настоящего приказа не подлежит.4. Государственному автономному учреждению города Москвы "Московская государственная экспертиза" при проведении государственной экспертизы проектной документации обеспечить контроль за соблюдением государственными заказчиками утвержденного Порядка (п.1).5. Контроль за исполнением настоящего приказа оставляю за собой.ПредседательИ.М.Солонников     Приложениек приказу Москомэкспертизыот 20 марта 2014 года N 29(В редакции, введенной в действие приказом Москомэкспертизы от 29 августа 2014 года N 80. - См. предыдущую редакцию)      Порядок определения нормативов затрат на содержание службы заказчика и осуществление строительного контроля при строительстве, реконструкции или капитальном ремонте объектов капитального строительства, финансирование которых осуществляется с привлечением средств бюджета города Москвы (с изменениями на 28 января 2022 года)1. Настоящий Порядок устанавливает правила определения нормативов затрат на содержание службы заказчика и осуществление строительного контроля при строительстве, реконструкции или капитальном ремонте объектов капитального строительства, финансирование которых осуществляется с привлечением средств бюджета города Москвы.2. Размер затрат для включения в главу 10 сводных сметных расчетов стоимости строительства "Содержание службы заказчика. Строительный контроль" определяется как сумма затрат на содержание службы заказчика и осуществление строительного контроля.3. При привязке типовых проектов затраты на содержание службы заказчика () устанавливаются в размере:- в базисном уровне цен по состоянию на 1 января 2000 г. - 1,25% от общей стоимости строительства по итогам глав 1-9 сводного сметного расчета (за исключением расходов, указанных в таблице 3), но не выше 931032 руб.;- в текущем уровне цен - 1,25% от общей стоимости строительства по итогам глав 1-9 сводного сметного расчета (за исключением расходов, указанных в таблице 3), но не выше величины, определенной по формуле:,где  - нормируемая среднемесячная заработная плата в строительстве в городе Москве в текущем уровне цен в рублях, утверждаемая Департаментом экономической политики и развития города Москвы.Распределение затрат по функциям и их размер (в долях от указанного размера затрат на содержание службы заказчика) приведены в таблице 1.(Пункт в редакции, введенной в действие приказом Москомэкспертизы от 28 января 2022 года N МКЭ-ОД/22-11. - См. предыдущую редакцию)4. Для индивидуальных проектов к установленным в п.3 Порядка размерам затрат применяется коэффициент 1,1.4.1. Для проектов по строительству, реконструкции, капитальному ремонту объектов Московского метрополитена норматив на содержание службы заказчика АО "Мосинжпроект" устанавливается в размере 2,98% от общей стоимости строительства по итогам глав 1-9 сводного сметного расчета (за исключением расходов, указанных в таблице 3).(Пункт 4.1 дополнительно включен приказом Москомэкспертизы от 15 декабря 2017 года N МКЭ-ОД/17-69)4.1.1. При необходимости учета дополнительных затрат, согласованных с Департаментом строительства города Москвы, на содержание службы заказчика по исполнению функций при проектировании, строительстве и вводе в эксплуатацию объектов Московского метрополитена, выполняемых Дирекцией строящегося метрополитена ГУП "Московский метрополитен", устанавливается лимит средств в размере не более 0,4% от общей стоимости строительства по итогам глав 1-9 сводного сметного расчета (за исключением расходов, указанных в таблице 3).(Пункт 4.1.1 дополнительно включен приказом Москомэкспертизы от 11 марта 2019 года N МКЭ-ОД/19-14)4.2. При необходимости учета дополнительных затрат по проведению предварительной экспертной оценки технических решений и стоимостных показателей для проектов по строительству и реконструкции объектов капитального строительства на территории города Москвы, а также проектов организации работ по сносу объектов капитального строительства, осуществляемого в рамках реализации Программы реновации жилищного фонда в городе Москве, согласованных главным распорядителем бюджетных средств в задании застройщика или технического заказчика на проектирование, либо обращения главного распорядителя бюджетных средств с подтверждением необходимости проведения предварительной экспертной оценки технических решений и стоимостных показателей размер затрат на предварительную экспертную оценку технических решений и стоимостных показателей определяется расчетом на основании проектных данных с применением Сборника 9.1 "Методика расчета стоимости научных, нормативно-методических, проектных и других видов работ (услуг) на основании нормируемых трудозатрат. МРР-9.1.02-18".(Пункт 4.2 дополнительно включен приказом Москомэкспертизы от 30 декабря 2019 года N МКЭ-ОД/19-95; в редакции, введенной в действие приказом Москомэкспертизы от 4 февраля 2021 года N МКЭ-ОД/21-7. - См. предыдущую редакцию)5. Нормативы затрат на осуществление строительного контроля определяются согласно таблице 2.Указанные нормативы учитывают выполнение контрольных мероприятий, проводимых заказчиком в рамках строительного контроля, перечень которых установлен пунктом 6 Положения о проведении строительного контроля при осуществлении строительства, реконструкции и капитального ремонта объектов капитального строительства, утвержденного постановлением Правительства Российской Федерации от 21 июня 2010 года N 468 "О порядке проведения строительного контроля при осуществлении строительства, реконструкции и капитального ремонта объектов капитального строительства".6. Показатель норматива затрат на осуществление строительного контроля определяется от суммы затрат по итогам глав 1-9 сводного сметного расчета (за исключением расходов, указанных в таблице 3) в базисном уровне цен.Для объектов, стоимость строительства которых по итогам глав 1-9 сводного сметного расчета (за исключением расходов, указанных в таблице 3) превышает 900 млн.руб. в базисном уровне цен, расчетное значение норматива затрат на осуществление строительного контроля округляется до четырех знаков после запятой.При определении затрат на осуществление строительного контроля норматив затрат применяется к сметной стоимости, определенной по итогам глав 1-9 сводного сметного расчета (за исключением расходов, указанных в таблице 3) в базисном и текущем уровне цен.(Пункт в редакции, введенной в действие приказом Москомэкспертизы от 28 января 2022 года N МКЭ-ОД/22-11. - См. предыдущую редакцию)7. Размер затрат на содержание службы заказчика и осуществление строительного контроля не содержит налога на добавленную стоимость (НДС). НДС учитывается за итогом сводного сметного расчета стоимости строительства.Таблица 1     Распределение затрат на содержание службы заказчика по функциям и их размер Nп/п Функция Доля от размера затрат на содержание службы заказчика, указанного в п.3 Порядка1 2 3 1 Составление запросов и обеспечение получения исходных данных и исходно-разрешительной документации, требуемых для подготовки проектной документации0,21 2 Сопровождение разработки, согласования проектной документации и прохождения государственной экспертизы0,19 3 Сопровождение разработки и согласования рабочей документации0,09 4 Подготовка и использование площадки строительства0,1 5 Подготовка исходной документации для проведения торгов на поставку оборудования и его передачи в эксплуатирующую организацию0,09 6 Подготовка документов для передачи объектов в собственность города Москвы и на баланс эксплуатирующим организациям0,32 В таблице 1 приведен полный комплекс затрат на содержание службы заказчика. Размер затрат включается в сметную документацию государственным заказчиком исходя из перечня функций, передаваемых техническому заказчику. Отнесение тех или иных функций к какому-либо комплексу затрат, указанных в графе 2, находится в компетенции государственного заказчика.(Пояснения в редакции, введенной в действие приказом Москомэкспертизы от 28 января 2022 года N МКЭ-ОД/22-11. - См. предыдущую редакцию)Таблица 2(В редакции, введенной в действие приказом Москомэкспертизы от 28 января 2022 года N МКЭ-ОД/22-11. - См. предыдущую редакцию) Нормативы затрат на осуществление строительного контроля Стоимость строительства по итогам глав 1-9 сводного сметного расчета в базисном уровне цен по состоянию на 1 января 2000 г. (за исключением расходов, указанных в таблице 3) (млн. рублей)Норматив затрат на осуществление строительного контроля (%)1 2 до 30 2,14 от 30 до 50 1,93 от 50 до 70 1,81 от 70 до 90 1,72 от 90 до 125 1,61 от 125 до 150 1,56 от 150 до 200 1,47 от 200 до 300 1,36 от 300 до 400 1,28 от 400 до 500 1,23 от 500 до 600 1,18 от 600 до 750 1,13 от 750 до 900 1,09 более 900 Норматив определяется по формуле:,где:Н - показатель норматива затрат на осуществление строительного контроля;С - стоимость строительства по итогам глав 1-9 сводного сметного расчета в базисном уровне цен по состоянию на 1 января 2000 г. (за исключением расходов, указанных в таблице 3); - стоимость строительства по итогам глав 1-9 сводного сметного расчета в базисном уровне цен по состоянию на 1 января 2000 г. (за исключением расходов, указанных в таблице 3), возведенная в степень 0,8022.          Таблица 3      Расходы, не подлежащие учету при определении нормативов затрат на содержание службы заказчика и осуществление строительного контроля Nп/п Наименование 1 2 1 Приобретение земельного участка2 Компенсация за вырубаемые зеленые насаждения и денежная форма компенсационного озеленения3 Технологическое присоединение к сетям инженерно-технического обеспечения4 Компенсация за сносимые строения и садово-огородные насаждения, возмещение убытков и потерь по переносу зданий и сооружений или строительству новых зданий и сооружений взамен сносимых, по возмещению убытков при отводе земель5 Денежная компенсация собственникам инженерных сетей и сооруженийРедакция документа с учетомизменений и дополнений подготовленаАО "Кодекс"   '
keyword = "объект"

sentences = find_sentences_with_word(text, keyword)
print(len(sentences))

print(len(text))
print("\n")
for s in sentences:
  print(s)

9
11509


Для проектов по строительству, реконструкции, капитальному ремонту объектов Московского метрополитена норматив на содержание службы заказчика АО "Мосинжпроект" устанавливается в

Для объектов, стоимость строительства которых по итогам глав 1-9 сводного сметного расчета (за

службы заказчика по исполнению функций при проектировании, строительстве и вводе в эксплуатацию объектов Московского метрополитена, выполняемых Дирекцией строящегося метрополитена ГУП "Московский метрополитен", устанавливается лимит

"О порядке проведения строительного контроля при осуществлении строительства, реконструкции и капитального ремонта объектов капитального строительства"

оборудования и его передачи в эксплуатирующую организацию0,09 6 Подготовка документов для передачи объектов в собственность города Москвы и на баланс эксплуатирующим организациям0,32 В таблице

Положения о проведении строительного контроля при осуществлении строительства, реконструкции и капитального ремонта объектов капитальн

In [ ]:
import pandas as pd

# Загрузка данных из Excel файла
df = pd.read_excel('proc.xlsx')

# Проверка и обновление значений
for index, row in df.iterrows():
    if pd.isna(row['Название НПА по стройконтролю']):
        df.at[index, 'Название НПА по стройконтролю'] = ""
        df.at[index, 'Ссылка на НПА по стройконтролю'] = ""
        df.at[index, 'объекты учета (стройконтроль)'] = ""
for index, row in df.iterrows():
    if pd.isna(row['Название НПА по стройнадзору']):
        df.at[index, 'Название НПА по стройнадзору'] = ""
        df.at[index, 'Ссылка на НПА по стройнадзору'] = ""
        df.at[index, 'объекты учета (стройнадзор)'] = ""

# Сохранение измененного DataFrame в Excel файл
df.to_excel('proc_updated.xlsx', index=False)

# Подтверждение успешного сохранения
print("Изменения сохранены в файле proc_updated.xlsx")

Изменения сохранены в файле proc_updated.xlsx
